In [62]:
#%%
import argparse
from dataloader import *
import torch
import torch.nn as nn?
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader  # For custom data-sets
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from models.deeplab import Deeplab
from utils import *
import time
from IPython import get_ipython
from file_utils import *

get_ipython().run_line_magic("load_ext", "autoreload")
get_ipython().run_line_magic("autoreload", "2")
import warnings
warnings.filterwarnings('ignore')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
def dict_to_args(d):

    args = argparse.Namespace()

    def dict_to_args_recursive(args, d, prefix=''):
        for k, v in d.items():
            if type(v) == dict:
                dict_to_args_recursive(args, v, prefix=k)
            elif type(v) in [tuple, list]:
                continue
            else:
                if prefix:
                    args.__setattr__(prefix + '_' + k, v)
                else:
                    args.__setattr__(k, v)

    dict_to_args_recursive(args, d)
    return args

In [64]:
config_dict = read_file('default.json')
config_dict

{'experiment_name': 'default_experiment',
 'version_number': '01',
 'train_data': './data/train_data.csv',
 'val_data': './data/val_data.csv',
 'test_data': './data/test_data.csv',
 'img_size': 256,
 'batch_size': 40,
 'num_workers': 8,
 'num_epochs': 50,
 'learning_rate': 0.001,
 'hidden_size': 512,
 'embedding_size': 300,
 'model_type': 'Baseline',
 'fine_tune': True}

In [53]:
# # TODO use a json to read these
# batch_size = 40
# learning_rate = 1e-3
# num_workers = 8
# epochs = 50
# fine_tune = True
# #%%

# #%%
# print(f"train size: {len(train_data)}")
# print(f"val size: {len(val_data)}")
# print(f"test size: {len(test_data)}")

In [54]:
class Experiment(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams
        print(self.hparams)
        self.fine_tune = self.hparams.fine_tune
        self.model = Deeplab(self.fine_tune)
        self.loss = nn.CrossEntropyLoss()
        self.lr = hparams.learning_rate
        self.batch_size = hparams.batch_size
        self.train_data = HypersimDataset(hparams.train_data)
        self.val_data = HypersimDataset(hparams.val_data)
        self.test_data = HypersimDataset(hparams.test_data)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        img, depth, semseg = batch
        output = self(img)
        loss = self.loss(output, semseg)
        self.log("train_loss", loss, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        img, depth, semseg = batch
        output = self(img)
        pred = output.argmax(1)
        loss = self.loss(output, semseg)
        IoUs = iou(pred, semseg, self.on_gpu)
        acc = pixel_acc(pred, semseg)
        return {"val_loss": loss, "val_ious": IoUs, "val_acc": acc}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        IoUs = torch.stack([x["val_ious"] for x in outputs], dim=0).sum(dim=0)
        acc = torch.stack([x["val_acc"] for x in outputs]).mean()
        mIoU = IoUs[0].sum() / IoUs[1].sum()
        self.log("val_loss", avg_loss, prog_bar=True)
        self.log("val_mIoU", mIoU, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def on_epoch_start(self):
        print("\n")

    def test_step(self, batch, batch_idx):
        img, depth, semseg = batch
        output = self(img)
        pred = output.argmax(1)
        loss = self.loss(output, semseg)
        IoUs = iou(pred, semseg, self.on_gpu)
        acc = pixel_acc(pred, semseg)
        return {"test_loss": loss, "test_ious": IoUs, "test_acc": acc}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        IoUs = torch.stack([x["test_ious"] for x in outputs], dim=0).sum(dim=0)
        acc = torch.stack([x["test_acc"] for x in outputs]).mean()
        mIoU = IoUs[0].sum() / IoUs[1].sum()
        self.log("test_loss", avg_loss)
        self.log("test_mIoU", mIoU)
        self.log("test_acc", acc)

    def configure_optimizers(self):
        optim = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        return optim

    def train_dataloader(self):
        train_loader = DataLoader(
            self.train_data,
            batch_size=self.batch_size,
            num_workers=num_workers,
            shuffle=True,
        )
        return train_loader

    def val_dataloader(self):
        val_loader = DataLoader(
            self.val_data, batch_size=self.batch_size, num_workers=num_workers, shuffle=False
        )
        return val_loader

    def test_dataloader(self):
        test_loader = DataLoader(
            self.test_data,
            batch_size=self.batch_size,
            num_workers=num_workers,
            shuffle=False,
        )
        return test_loader

In [70]:
tb_logger = pl.loggers.TensorBoardLogger(save_dir='./',
                                         name=f'{str(config_dict["experiment_name"])}', # This will create different subfolders for your models
                                         version=f'v_{str(config_dict["version_number"])}') 

In [ ]:
#%%
exp = Experiment(dict_to_args(config_dict))
trainer = pl.Trainer(
    fast_dev_run=False,
    gpus=1,
    max_epochs=epochs,
    progress_bar_refresh_rate=5,
    logger=tb_logger,
)
#%%
trainer.tune(exp)
#%%
trainer.fit(exp)
# trainer.test(exp)
#%%

"batch_size":      40
"embedding_size":  300
"experiment_name": default_experiment
"fine_tune":       True
"hidden_size":     512
"img_size":        256
"learning_rate":   0.001
"model_type":      Baseline
"num_epochs":      50
"num_workers":     8
"test_data":       ./data/test_data.csv
"train_data":      ./data/train_data.csv
"val_data":        ./data/val_data.csv
"version_number":  01


GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name  | Type             | Params
-------------------------------------------
0 | model | Deeplab          | 59.3 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
59.3 M    Trainable params
0         Non-trainable params
59.3 M    Total params
237.396   Total estimated model params size (MB)
